In [1]:
using PyPlot

INFO: Loading help data...


In [47]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
#PyPlot.hold(false)

(fig, ax) = plt.subplots(8,8)
x=[1:1:100]
y=[rand(100)]
li=[hi[:plot](x, y)[1] for hi in ax]

fig[:canvas][:draw]()
plt.show()

y2=[rand(500,100)]

@time for i=1:10
    
    #[fig[:canvas][:restore_region](hi) for hi in backgrounds]
    [hi[:set_ydata](y2[i,:]) for hi in li]
    [fig[:canvas][:blit](hi[:bbox]) for hi in ax]
    #fig[:canvas][:draw]()
end

elapsed time: 1.072894188 seconds (471040 bytes allocated)


In [1]:
addprocs();
function sendto(p::Int; args...)
    for (nm, val) in args
        @spawnat(p, eval(Main, Expr(:(=), nm, val)))
    end
end


function sendto(ps::Vector{Int}; args...)
    for p in ps
        sendto(p; args...)
    end
end

getfrom(p::Int, nm::Symbol; mod=Main) = fetch(@spawnat(p, getfield(mod, nm)))

#create local copies for each processor
sendto(2, s=Any[SpikeDetection() for i=1:32])
sendto(3, s=Any[SpikeDetection() for i=1:32])
sendto(2, c=Any[Cluster() for i=1:32])
sendto(3, c=Any[Cluster() for i=1:32])

#this is read only, so should work fine, but might have to be shared array
be=ones(Int64,20000,64)
for i=1:64
    be[200:700:end,i]=10*be[200:700:end,i]
end

#each processor should run independently
function main()
for p in workers()
    @spawnat p begin
        onlineCal_par(be,c,s,p-1,2)
    end
end
end

getfrom (generic function with 1 method)

In [2]:
using SortSpikes, ExtractSpikes, DistributedArrays

In [4]:

s=Any[SpikeDetection() for i=1:64]
c=Any[Cluster() for i=1:64]
electrode=Any[zeros(Int64,2) for i=1:64]
neuronnum=Any[zeros(Int64,2) for i=1:64]
be=ones(Int64,20000,64)
for i=1:64
    be[200:700:end,i]=10*be[200:700:end,i]
end
@time onlineCal(be,c,s)
onlineSort([1,20000],be,c,s,electrode,neuronnum)



 137.937 milliseconds (101 k allocations: 36699 KB, 6.42% gc time)


In [38]:
function cwt(x::Array{Float64,1},t::Int,psi::Array{Float64,1},bs::Array{Float64,1})

    bs[1]=bs[2]-x[t-20]*psi[1]+x[t]*psi[21]

    for i=2:19
        bs[i]=bs[i+1]-x[t-20]*psi[i]+x[t]*psi[22-i] 
    end

    bs[20]=0.0
    for i=1:20
        bs[20]+=x[t-20+i]*psi[21-i]
    end
    
    return bs
end


function mycwt(x::Array{Float64,1},t::Int, psi::Array{Float64,1})
    
    bs=0.0
    
    for i=1:20
        bs+=x[t-20+i]*psi[21-i]
    end
    
end

function main()
    for i=1:20000
        mycwt(x,t,psi)
    end
end

psi=[0.00021208083980379827
0.00035858968789573785
-0.0021782363581090178
-0.004159358781386048
0.010131117519849788
0.023408156785839195
-0.02816802897093635
-0.09192001055969624
0.05204316317624377
0.4215662066908515
-0.7742896036529562
0.4379916261718371
0.06203596396290357
-0.10557420870333893
-0.0412892087501817
0.03268357426711183
0.01976177894257264
-0.009164231162481846
-0.006764185448053083
0.0024333732126576722
0.0016628637020130838
-0.0006381313430451114
-0.00030225958181306315
0.00014054114970203437
4.134043227251251e-05
-2.1315026809955787e-05
-3.7346551751414047e-06
2.0637618513646814e-06
1.6744288576823017e-07
-9.517657273819165e-08
];

bs=zeros(Float64,20)
    
x=rand(100)
t=50

for j=1:20
    for i=1:20
        bs[j]+=x[t-20+i]*psi[21-i]
    end
end

main (generic function with 1 method)

In [1]:
addprocs()
@everywhere using SortSpikes, ExtractSpikes, DistributedArrays

In [13]:
st=Array(Sorting,64)
for i=1:64
    st[i]=Sorting(SpikeDetection(),Cluster(),ones(Int64,20000))
    st[i].rawSignal[200:700:end]=10*st[i].rawSignal[200:700:end]
end
s=distribute(st);

In [14]:
@everywhere function add_one!(mt::Sorting)
    onlineCal(mt)
    return mt
end

@time map!(add_one!, s);

 112.952 milliseconds (1295 allocations: 104 KB)
